In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D


# Own utility methods:
from mandelbrot import *
from sampling_methods import *

# 1.1
We create an image of the mandelbrot set by applying the mandelbrot iteration 1000 times to a grid of complex numbers and plotting the number of iterations until divergence for each pixel.

In [ ]:

def plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as = 'mandelbrot_visualisation'):
    plt.imshow(num_div_steps, cmap=cmap)
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.colorbar(label='Divergence Steps')
    plt.title('Mandelbrot Set Divergence Visualization')

    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    plt.show()
    
    

resolution = 1000
C = generate_complex_grid(resolution)
num_div_steps, _ = compute_mandelbrot_torch(C, max_steps=500, bound=10)
plot_mandelbrot(num_div_steps)

We also zoom into the Mandelbrot set revealing the  intricate, self-similar patterns, enhancing visual understanding of its fractal nature.

The names of the parts we zoom into:
- Seahorse Valley
- Elephants Valley
- Spiral Region

In [ ]:
# Modify these parameters for zooming
zoom_center = (-0.75, 0.1)  # Seahorse Valley
zoom_level = 0.1  # Smaller values for deeper zooms

# Compute ranges for the zoomed-in area
real_range = (zoom_center[0] - zoom_level, zoom_center[0] + zoom_level)
imag_range = (zoom_center[1] - zoom_level, zoom_center[1] + zoom_level)

# Generate the grid with the new zoomed-in ranges
resolution = 1000
C = generate_complex_grid(resolution, real_range=real_range, imag_range=imag_range)

# Compute the Mandelbrot set with a high number of steps for detailed structure
num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=1000, bound=10)

# Plot the result
plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as= 'visualisation_seahorse')  

In [ ]:
# Modify these parameters for zooming
zoom_center = (0.3, -0.05) # Elephants Valley
zoom_level = 0.05  # Smaller values for deeper zooms 

# Compute ranges for the zoomed-in area
real_range = (zoom_center[0] - zoom_level, zoom_center[0] + zoom_level)
imag_range = (zoom_center[1] - zoom_level, zoom_center[1] + zoom_level)

# Generate the grid with the new zoomed-in ranges
resolution = 1000
C = generate_complex_grid(resolution, real_range=real_range, imag_range=imag_range)

# Compute the Mandelbrot set with a high number of steps for detailed structure
num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=1000, bound=10)

# Plot the result
plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as= 'visualisation_elephant') 

In [ ]:
# Modify these parameters for zooming
zoom_center = (-0.7, 0.3) # Spiral Region
zoom_level = 0.05  # Smaller values for deeper zooms 

# Compute ranges for the zoomed-in area
real_range = (zoom_center[0] - zoom_level, zoom_center[0] + zoom_level)
imag_range = (zoom_center[1] - zoom_level, zoom_center[1] + zoom_level)

# Generate the grid with the new zoomed-in ranges
resolution = 2000
C = generate_complex_grid(resolution, real_range=real_range, imag_range=imag_range)

# Compute the Mandelbrot set with a high number of steps for detailed structure
num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=1000, bound=10)

# Plot the result
plot_mandelbrot(num_div_steps, cmap='viridis', save_plot_as= 'visualisation_spiral') 

# 1.2
Convergence of the Area estimate with the number of iterations

In [ ]:

def plot_mandelbrot_area_difference(sample_sizes, iteration_count, skip_iterations = 0, save_plot_as = 'mandelbrot-convergence-iterations.png'):
    area_estimates = np.zeros((len(sample_sizes), iteration_count))

    for i, num_samples in enumerate(sample_sizes):
        C = uniform_random_sampling(num_samples, (-2,2),(-2,2))
        _, area_est = compute_mandelbrot_torch(C, iteration_count, area_factor=16 )
        area_estimates[i,:] = area_est

    plt.figure(figsize=(10, 5))
    for i, samples in enumerate(sample_sizes):
        plt.plot(np.arange(iteration_count)[skip_iterations:]+1, np.abs(area_estimates[i,-1] - area_estimates[i, skip_iterations:]), label=f's = {samples}')

    plt.xlabel('Iteration j')
    plt.ylabel(r'$A_{j, s} - A_{i, s}$')
    plt.title('Convergence of Estimated Area depending on Iteration Count')
    plt.legend()
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True)
    
    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    plt.show()

    return area_estimates




# Parameters used for the plot in the report (runs for over 1 hour):
# sample_sizes = [1000, 10000, 100000, 1000000]
# iteration_count = 5000000

# example with lower run time
sample_sizes = [1000, 10000, 100000]
iteration_count = 10000

plot_mandelbrot_area_difference(sample_sizes, iteration_count)

Hoeffding's inequality:

for a series of independent random variables $X_i \in [0,1]$  writing $S_n = \frac1n\sum_{i=1}^n X_i$
\begin{align}
     P(|S_n - E[S_n]| \geq \varepsilon) \leq 2 exp \left(-2\varepsilon^2 n\right) &\leq \delta\\
     {2\varepsilon^2n} &\leq -log(\delta / 2) \\
    \varepsilon &\leq \sqrt{-\frac 1{2n} log(\delta / 2)}
\end{align}

Convergence with number of samples

In [ ]:
def plot_mandelbrot_area_difference_sample_count(sample_sizes, iteration_count, sampling_methods, method_titles, save_plot_as = 'images/mandelbrot-convergence-samples.png'):
    area_estimates = np.zeros((len(sampling_methods), len(sample_sizes)))

    for m, method in enumerate(sampling_methods):
        for i, num_samples in enumerate(sample_sizes):
            C = method(num_samples, (-2,2),(-2,2))
            _, area_est = compute_mandelbrot_torch(C, iteration_count, area_factor=16 )
            area_estimates[m, i] = area_est[-1]



    plt.figure(figsize=(10, 5))
    
    markers=['o', 'v', '^','s', 'p' ]   

    for m, method in enumerate(sampling_methods):
        plt.plot(sample_sizes[:-1], np.abs(area_estimates[m, :-1] - area_estimates[m, -1]), label=method_titles[m], marker = markers[m] )

    plt.xlabel('Sample Size s')
    plt.ylabel(r'$A_{i, s} - A_{i, s_{max}}$')
    plt.title('Convergence of Estimated Area depending on Sample Size')
    plt.legend()
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(True)
    
    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    plt.show()

    return area_estimates



sample_sizes = np.logspace(3, 7, 20, base=10).astype(np.int64)
iteration_count = 10000
sampling_methods = [ uniform_random_sampling, latin_hypercube_sampling,  orthogonal_sampling]
method_titles = [ 'uniform random', 'latin hypercube', 'orthogonal']
area_estimates = plot_mandelbrot_area_difference_sample_count(sample_sizes, iteration_count, sampling_methods, method_titles)
print(area_estimates)

# 1.3
In this part we look at different sampling methods to estimate the size of the Mandelbrot set to compare.


 *Pure Random Sampling*

In [ ]:
def plot_sampling_method(C, title, max_steps=1000, area_factor=9, save_plot_as = 'sampling_method'):
    """
    Computes and plots Mandelbrot set divergence steps for a given complex array using a specified sampling method.

    Parameters:
    - C: 1D array of complex numbers representing the samples.
    - title: Title for the plot.
    - max_steps: Maximum number of iterations for Mandelbrot computation.
    - area_factor: Area factor for the computation.
    """
    num_div_steps, area_at_step = compute_mandelbrot(C, max_steps=max_steps, area_factor=area_factor)

    rval, ival = C.real, C.imag
    plt.scatter(rval, ival, s=0.1, alpha=0.1, c=num_div_steps)
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.title(title)
    plt.colorbar(label='Divergence Steps')

    full_path = os.path.join('plots', save_plot_as)
    plt.savefig(full_path, dpi=600)
    plt.show()

def pure_random_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=42):
    """
    Performs pure random sampling for the Mandelbrot set.

    Parameters:
    - num_samples: Number of samples.
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - C: 1D array of complex numbers representing the samples.
    """
    np.random.seed(seed)
    rval = np.random.uniform(real_range[0], real_range[1], num_samples)
    ival = np.random.uniform(imag_range[0], imag_range[1], num_samples)
    return rval + 1.j * ival

# Example Usage
C = pure_random_sampling(num_samples=100000)
plot_sampling_method(C, "Mandelbrot Set Divergence Steps (Pure Random Sampling)", save_plot_as = 'pure_random_sampling')

*Orthogonal Sampling*

In [ ]:
def orthogonal_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=42):
    """
    Performs orthogonal sampling for the Mandelbrot set, suited for Python while following the logic of the provided C code.

    Parameters:
    - num_samples: Total number of samples to generate (must be a perfect square).
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - C: 1D array of complex numbers representing the samples.
    """
    np.random.seed(seed)
    major = int(np.round(np.sqrt(num_samples)))
    num_samples = major * major
    print(f"Adjusted num_samples to {num_samples} to ensure it is a perfect square close to the original value.")

    x_indices = np.arange(major)
    y_indices = np.arange(major)

    samples = []
    for i in range(major):
        np.random.shuffle(x_indices)
        np.random.shuffle(y_indices)

        for j in range(major):
            rand_real = np.random.uniform(0, 1)
            rand_imag = np.random.uniform(0, 1)

            x = real_range[0] + (real_range[1] - real_range[0]) * ((i + rand_real) / major)
            y = imag_range[0] + (imag_range[1] - imag_range[0]) * ((j + rand_imag) / major)

            samples.append(complex(x, y))

    return np.array(samples)


# Example Usage
C = orthogonal_sampling(num_samples=100000)
plot_sampling_method(C, "Mandelbrot Set Divergence Steps (Orthogonal Sampling)", save_plot_as='orthogonal_sampling')

*Latin HyperCube Sampling*

In [ ]:
def latin_hypercube_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=42):
    """
    Performs Latin Hypercube sampling for the Mandelbrot set.

    Parameters:
    - num_samples: Number of samples.
    - real_range: Tuple specifying the range for the real axis.
    - imag_range: Tuple specifying the range for the imaginary axis.
    - seed: Random seed for reproducibility.

    Returns:
    - C: 1D array of complex numbers representing the samples.
    """
    np.random.seed(seed)
    sampler = LatinHypercube(d=2)
    lhs_samples = sampler.random(n=num_samples)

    scaled_samples = scale(lhs_samples, [real_range[0], imag_range[0]], [real_range[1], imag_range[1]])
    rval, ival = scaled_samples[:, 0], scaled_samples[:, 1]
    return rval + 1.j * ival

# Example Usage
C = latin_hypercube_sampling(num_samples=100000)
plot_sampling_method(C, "Mandelbrot Set Divergence Steps (Latin Hypercube Sampling)", save_plot_as = 'latin_hypercube_sampling')

# 1.4

In [ ]:
def is_in_mandelbrot(c, max_iter):
    """
    Determines if a complex number is in the Mandelbrot set.

    Parameters:
    - c: Complex number.
    - max_iter: Maximum number of iterations to check.

    Returns:
    - Boolean indicating whether the point is in the Mandelbrot set.
    """
    z = 0
    for n in range(max_iter):
        z = z*z + c
        if abs(z) > 2:
            return False
    return True

def estimate_area_mandelbrot(num_samples, max_iter, real_range=(-3, 1), imag_range=(-2, 2)):
    """
    Estimates the area of the Mandelbrot set using Monte Carlo sampling.

    Parameters:
    - num_samples: Number of random samples.
    - max_iter: Maximum number of iterations for Mandelbrot check.
    - real_range: Range of real values (tuple).
    - imag_range: Range of imaginary values (tuple).

    Returns:
    - Estimated area of the Mandelbrot set.
    """
    real = np.random.uniform(real_range[0], real_range[1], num_samples)
    imag = np.random.uniform(imag_range[0], imag_range[1], num_samples)
    complex_samples = real + 1j * imag

    in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])

    rect_area = (real_range[1] - real_range[0]) * (imag_range[1] - imag_range[0])
    area_estimate = np.mean(in_set) * rect_area
    return area_estimate

### Adaptive sampling

In [ ]:
# We are using Adaptive Sampling, a method used to iteratively improve the estimation of an area by focusing on regions with higher variance.
# The idea behind adaptive sampling is to refine the sampling process over multiple iterations,
# allowing the method to adapt and focus more on areas that are likely to contribute more to the accuracy of the final result.

true_area = 1.5063

# Function for Importance Sampling
def importance_sampling(num_samples, max_iter, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=None):
    if seed is None:
        seed = random.randint(0, 10000)
    np.random.seed(seed)

    mean_real = 0
    mean_imag = 0
    std_dev = 0.8

    real_samples = np.random.normal(mean_real, std_dev, num_samples)
    imag_samples = np.random.normal(mean_imag, std_dev, num_samples)

    real_samples = np.clip(real_samples, real_range[0], real_range[1])
    imag_samples = np.clip(imag_samples, imag_range[0], imag_range[1])

    complex_samples = real_samples + 1j * imag_samples

    proposal_density = (1 / (2 * np.pi * std_dev**2)) * np.exp(-(real_samples**2 + imag_samples**2) / (2 * std_dev**2))

    in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])

    target_density = 1 / ((real_range[1] - real_range[0]) * (imag_range[1] - imag_range[0]))

    weights = target_density / proposal_density

    rect_area = (real_range[1] - real_range[0]) * (imag_range[1] - imag_range[0])
    weighted_area_estimate = np.mean(in_set * weights) * rect_area

    return weighted_area_estimate

# Function for Sobol Sampling (Quasi-Monte Carlo)
def sobol_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=None):
    if seed is None:
        seed = random.randint(0, 10000)
    np.random.seed(seed)
    sampler = Sobol(d=2, scramble=True)
    sobol_samples = sampler.random(n=num_samples)

    scaled_samples = scale(sobol_samples, [real_range[0], imag_range[0]], [real_range[1], imag_range[1]])

    rval, ival = scaled_samples[:, 0], scaled_samples[:, 1]
    return rval + 1.j * ival

# Function for Adaptive Sampling
def adaptive_sampling(num_samples, max_iter, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=None, iterations=10):
    if seed is None:
        seed = random.randint(0, 10000)
    np.random.seed(seed)

    rval = np.random.uniform(real_range[0], real_range[1], num_samples)
    ival = np.random.uniform(imag_range[0], imag_range[1], num_samples)
    complex_samples = rval + 1.j * ival

    in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])
    rect_area = (real_range[1] - real_range[0]) * (imag_range[1] - imag_range[0])
    area_estimate = np.mean(in_set) * rect_area

    for _ in range(iterations):
        variance = np.var(in_set)

        num_refine_samples = int(num_samples * 0.5)  # Increase sample size
        refine_rval = np.random.uniform(real_range[0], real_range[1], num_refine_samples)
        refine_ival = np.random.uniform(imag_range[0], imag_range[1], num_refine_samples)

        refine_samples = refine_rval + 1.j * refine_ival
        complex_samples = np.concatenate((complex_samples, refine_samples))

        in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])

        area_estimate = np.mean(in_set) * rect_area

    return area_estimate

def run_simulations(sample_sizes, max_iter=1000):
    errors_random = []
    errors_importance = []
    errors_sobol = []
    errors_adaptive = []

    for num_samples in sample_sizes:
        seed = random.randint(0, 10000)

        # Normal Monte Carlo (Random Sampling)
        C_random = pure_random_sampling(num_samples, seed=seed)
        area_random = estimate_area_mandelbrot(C_random, max_iter)  # Pass C_random instead of len(C_random)
        error_random = abs(area_random - true_area)
        errors_random.append(error_random)

        # Importance Sampling
        area_importance = importance_sampling(num_samples, max_iter, seed=seed)
        error_importance = abs(area_importance - true_area)
        errors_importance.append(error_importance)

        # Sobol Sampling (Quasi-Monte Carlo)
        C_sobol = sobol_sampling(num_samples, seed=seed)
        area_sobol = estimate_area_mandelbrot(C_sobol, max_iter)  # Pass C_sobol instead of len(C_sobol)
        error_sobol = abs(area_sobol - true_area)
        errors_sobol.append(error_sobol)

        # Adaptive Sampling
        area_adaptive = adaptive_sampling(num_samples, max_iter, seed=seed)
        error_adaptive = abs(area_adaptive - true_area)
        print(error_adaptive)
        errors_adaptive.append(error_adaptive)

    return errors_random, errors_importance, errors_sobol, errors_adaptive

In [ ]:


# Refactored AdaptiveSampler class
class AdaptiveSampler:
    def __init__(self, num_samples, max_iter, real_range=(-2, 1), imag_range=(-1.5, 1.5),
                 seed=None, error_threshold=1e-4, max_iterations=10):
        self.num_samples = num_samples
        self.max_iter = max_iter
        self.real_range = real_range
        self.imag_range = imag_range
        self.seed = seed if seed is not None else random.randint(0, 10000)
        self.error_threshold = error_threshold
        self.max_iterations = max_iterations

        np.random.seed(self.seed)

        self.initial_grid_size = max(1, int(np.sqrt(self.num_samples)))
        self.initial_samples_per_region = max(1, self.num_samples // (self.initial_grid_size ** 2))
        self.regions = self.create_grid()

    class Region:
        def __init__(self, x_min, x_max, y_min, y_max):
            self.x_min = x_min
            self.x_max = x_max
            self.y_min = y_min
            self.y_max = y_max
            self.samples = []
            self.in_set_counts = []
            self.total_samples = 0
            self.local_area_estimate = 0
            self.local_variance = np.inf

        def sample(self, num_samples, max_iter):
            x_samples = np.random.uniform(self.x_min, self.x_max, num_samples)
            y_samples = np.random.uniform(self.y_min, self.y_max, num_samples)
            complex_samples = x_samples + 1j * y_samples
            in_set = np.array([is_in_mandelbrot(c, max_iter) for c in complex_samples])
            self.samples.extend(complex_samples)
            self.in_set_counts.extend(in_set)
            self.total_samples += num_samples
            area = (self.x_max - self.x_min) * (self.y_max - self.y_min)
            p = np.mean(self.in_set_counts)
            self.local_area_estimate = p * area
            if self.total_samples > 0:
                self.local_variance = (p * (1 - p) / self.total_samples) * area ** 2
            else:
                self.local_variance = np.inf

        def increase_sample_count(self, num_samples, max_iter):
            self.sample(num_samples, max_iter)

    def create_grid(self):
        x_min, x_max = self.real_range
        y_min, y_max = self.imag_range
        x_edges = np.linspace(x_min, x_max, self.initial_grid_size + 1)
        y_edges = np.linspace(y_min, y_max, self.initial_grid_size + 1)
        regions = []
        for i in range(self.initial_grid_size):
            for j in range(self.initial_grid_size):
                region = self.Region(x_edges[i], x_edges[i + 1], y_edges[j], y_edges[j + 1])
                regions.append(region)
        return regions

    def combine_region_estimates(self):
        total_area_estimate = sum(region.local_area_estimate for region in self.regions)
        return total_area_estimate

    def estimate_overall_variance(self):
        total_variance = sum(region.local_variance for region in self.regions)
        return total_variance

    def select_regions_with_high_variance(self, fraction=0.5):
        variances = np.array([region.local_variance for region in self.regions])
        threshold = np.percentile(variances, 100 * (1 - fraction))
        regions_to_refine = [region for region in self.regions if region.local_variance >= threshold]
        return regions_to_refine

    def run(self):
        for iteration in range(self.max_iterations):
            # Sample in each region
            for region in self.regions:
                if region.total_samples == 0:
                    samples_to_draw = self.initial_samples_per_region
                else:
                    samples_to_draw = max(1, self.initial_samples_per_region // (iteration + 1))
                region.sample(samples_to_draw, self.max_iter)
            # Combine estimates
            area_estimate = self.combine_region_estimates()
            # Estimate variance
            variance_estimate = self.estimate_overall_variance()
            # Check for convergence
            if variance_estimate < self.error_threshold:
                break
            # Select regions to refine
            regions_to_refine = self.select_regions_with_high_variance()
            if not regions_to_refine:
                break
            additional_samples = max(1, self.initial_samples_per_region // (iteration + 1))
            for region in regions_to_refine:
                region.increase_sample_count(additional_samples, self.max_iter)
        return area_estimate


def run_simulations(sample_sizes, max_iter=1000):
    errors_random = []
    errors_adaptive = []
    errors_orthogonal = []

    for num_samples in sample_sizes:
        seed = random.randint(0, 10000)

        # Normal Monte Carlo (Random Sampling)
        C_random = pure_random_sampling(num_samples, seed=seed)
        area_random = estimate_area_mandelbrot(C_random, max_iter)
        error_random = abs(area_random - true_area)
        errors_random.append(error_random)

        # Orthogonal Sampling
        C_orthogonal = orthogonal_sampling(num_samples, real_range=(-2, 1), imag_range=(-1.5, 1.5), seed=seed)
        area_orthogonal = estimate_area_mandelbrot(C_orthogonal, max_iter)
        error_orthogonal = abs(area_orthogonal - true_area)
        errors_orthogonal.append(error_orthogonal)

        # Adaptive Sampling
        adaptive_sampler = AdaptiveSampler(num_samples, max_iter, seed=seed)
        area_adaptive = adaptive_sampler.run()
        error_adaptive = abs(area_adaptive - true_area)
        errors_adaptive.append(error_adaptive)

    return errors_random, errors_orthogonal, errors_adaptive

min_sample_size = 1000
max_sample_size = 1000000
sample_sizes = np.logspace(np.log10(min_sample_size), np.log10(max_sample_size), num=29).astype(int)

errors_random, errors_orthogonal, errors_adaptive = run_simulations(sample_sizes)

# Calculate mean and standard deviation
mean_orthogonal = np.mean(errors_orthogonal)
std_orthogonal = np.std(errors_orthogonal)
mean_adaptive = np.mean(errors_adaptive)
std_adaptive = np.std(errors_adaptive)

# Statistical comparison
t_stat, p_value = stats.ttest_ind(errors_orthogonal, errors_adaptive)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

# Print results
print("Statistical Summary:")
print(f"Orthogonal Sampling: Mean = {mean_orthogonal:.4f}, Std Dev = {std_orthogonal:.4f}")
print(f"Adaptive Sampling: Mean = {mean_adaptive:.4f}, Std Dev = {std_adaptive:.4f}")
print("\nT-tests Results:")
print(f"Orthogonal vs Adaptive: t-statistic = {t_stat:.4f}, p-value = {p_value:.4g}")

# Plotting results
plt.figure(figsize=(8, 6))
plt.loglog(sample_sizes, errors_adaptive, marker='o', label="Adaptive Sampling")
plt.loglog(sample_sizes, errors_orthogonal, marker='o', label="Orthogonal Sampling")
plt.xlabel('Number of Samples')
plt.ylabel('Error |A_est - A_true|')
plt.title('Comparison of Convergence Rates for Monte Carlo Methods')
plt.legend()
plt.grid(True)
plt.show()